# Homework

In this homework, we'll deploy the churn preduction model from session 5. We already have a docker image for this model - we'll use it for deploying the model to Kubernetes.

# Building the Image
Clone the course repo if you haven't:  ```
git clone https://github.com/alexeygrigorev/mlbookcamp-code.git```

Go to the ```course-zoomcamp/05-deployment/code``` folder and execute the following: ```
docker build -t churn-model:v001 .
```

Run it to test that it's working locally:
```docker run -it --rm -p 9696:9696 churn-model:v001```

And in another terminal, execute ```predict-test.py``` file:

Output: ```{'churn': False, 'churn_probability': 0.3257561103397851}
not sending promo email to xyz-123```

Test worked

Now you can stop the container running in Docker.

# Installing kubectl and kind

* kubectl - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* kind - https://kind.sigs.k8s.io/docs/user/quick-start/

# Quesion 1: Version of kind

What's the version of kind that you have?

Use ```kind --version``` to find out.

**Answer:** kind version 0.12.0

# Creating a Cluster

Now let's create a cluster with kind:

```kind create cluster```

# Question 2: Verifying that everything works
Now let's test if everything works. Use ```kubectl``` to get the list of running services.

What's ```CLUSTER-IP``` of the service that is already running there?

**Answer:** 10.96.0.1

# Question 3: Uploading the image to kind

To be able to use the docker image we previously created (churn-model:v001), we need to register it with kind.

What's the command we need to run for that?

**Answer:** kind load docker-image ```churn-model:v001```

# Question 4: Creating a deployment

Now let's create a deployment (e.g. ```deployment.yaml```):

```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: churn
spec:
  selector:
    matchLabels:
      app: churn
  template:
    metadata:
      labels:
        app: churn
    spec:
      containers:
      - name: churn
        image: <Image>
        resources:
          limits:
            memory: "128Mi"
            cpu: "500m"
        ports:
        - containerPort: <Port>
```

**Answer:** The port is ```9696``` and the image is ```churn-model:v001```

# Question 5: Pod name

Apply this deployment:

```kubectl apply -f deployment.yaml```

Now get a list of running pods. What's the name of the pod that just started?

**Answer:** Use ```kubectl get pods```. The name is ```churn-5f694c8bbb-4vjxr```

# Question 6: Creating a service

Let's create a service for this deployment (```service.yaml```):

```
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

**Answer:**
* ```<Service name> = churn```
* ```<???> = churn```
* ```<PORT> = 9696```

Apply the config file: ```kubectl apply -f service.yaml```

# Testing the service locally

We can do it by forwarding the 9696 port on our computer to the port 80 on the service:

```kubectl port-forward service/churn 9696:80```

Run ```predict-test.py``` from session 5 to verify that everything is working.

Output: ```{'churn': False, 'churn_probability': 0.469135109804453}```